In [18]:
#Python script that collects top 5 popular videos of every user that ranked in top 100 and post data of his timeline
import re
import json
import sys
import os
import pickle
import shutil


Root = "/datasets/sagarj/vineData/Dataset/"

videoDir = Root+"Videos/"
postDir = Root+"Posts/"
sampled = Root+"sampledFrames/"



In [19]:
def getVisited(visitedList):
    visited = []
    try:
        f = open(visitedList, 'rb')
        visited = pickle.load(f)
    except IOError:
        f = open(visitedList,"a+")
        pickle.dump([],f)
    return visited


def updateVisited(visited,visitedList):
    with open(visitedList, 'wb') as f:
        pickle.dump(visited, f)


def walkLevel1Dir(root):
    count = 0
    dirList = []
    filesList = []
    for path, dirs, files in os.walk(root):
        if count > 0:
            return dirList , fileList
        dirList = dirs
        fileList = files
        count = count + 1
        
def listFiles(root):
    dirList = []
    filesList = []
    for path, dirs, files in os.walk(root):
        return dirs , files


def getPopularFile(rootDir):
    f = open(rootDir + '/popular.json' ,'r')
    data = json.load(f)
    return data

def getJson(jsonFile):
    f = open(jsonFile ,'r')
    data = json.load(f)
    return data

# Get Top 3 liked videos
def getVideos(usrProf, userId, vid ):
    videoDir = vid + "/" + str(userId)
    print "Working on Vids:  "  + videoDir
    if not os.path.exists(videoDir):
        os.makedirs(videoDir)
    if len(usrProf) < 2:
        return
    if not isinstance(usrProf['data'],dict):
        print "JSON Data not a dict"
        return
    records = usrProf['data']['records']
    likes = []
    for record in records:
        likes.append(record['likes']['count'])
    likes.sort(reverse=True)
    
    for record in records:
        if record['likes']['count'] in likes[:3]:
            videoString = record['videoDashUrl']
            if videoString:
                videoUrl = record['videoDashUrl'].split('?');
                argsVideo = ['wget', '-r', '-l', '1', '-p', '-P' ,'-nd', videoDir, videoUrl[0]]
                call(argsVideo);

def getPostMeta(usrProf, userId , post ):
    postDir = post + "/" + str(userId)
    print "Working on Post:  "  + postDir
    if not os.path.exists(postDir):
        os.makedirs(postDir)
    if not isinstance(usrProf['data'],dict):
        print "JSON Data not a dict"
        return
    records = usrProf['data']['records']
 
    for subRecord in records:
        postID = subRecord['postId']
        postData = requests.get("https://api.vineapp.com/timelines/posts/"+str(postID))
        post = open( postDir + "/" + str(postID) + '.json', 'w')
        json.dump(postData.json(), post)

def getHistory(popular , dataRoot):
    records = popular['data']['records']
    timeline = dataRoot + "/" + userPastVideos
    posts = dataRoot + "/" + userPastVideoMeta
    if not os.path.exists(timeline):
        os.makedirs(timeline)
    if not os.path.exists(posts):
        os.makedirs(posts)
    print "now in : " + dataRoot
    for record in records:
        userId = record['userId']
        profile_file = dataRoot + "/userTimeline/" + str(userId) + ".json"
        print "Working on file:  "  + profile_file
        if os.path.exists(profile_file):
            usrProf = getJson(profile_file)
            getVideos(usrProf,userId, timeline)
            getPostMeta(usrProf,userId,posts)

def getList(textFile):
    lines = []
    ids = []
    with open(textFile) as f:
        lines = f.readlines()
    for line in lines:
        parts = line.split('/')
        ids.append(parts[-1])
    unique = list(set(ids))
    print len(unique)
    ulines = []
    for i in unique:
        ulines.append(lines[ids.index(i)].strip())
    print "%d reduced to %d"%(len(lines), len(ulines)) 
    return ulines

def copyFile(src, dest):
    if not os.path.exists(src): 
        return
    try:
        shutil.copy(src, dest)
    # eg. src and dest are the same file
    except shutil.Error as e:
        print('Error: %s' % e)
    # eg. source or destination doesn't exist
    except IOError as e:
        print('Error: %s' % e.strerror)

In [20]:
d , posts = listFiles(postDir)
d , vids = listFiles(videoDir)
d , samples = listFiles(sampled)

In [27]:
print posts[1]

1270239341221961728.json


In [25]:
sampledPosts = []
for s in samples:
    post = s.strip().split('_')[0]
    if post not in sampledPosts:
        sampledPosts.append(post)

In [26]:
print len(sampledPosts)

6752


In [28]:
#postList = getList(posts)

In [ ]:
# print allPosts[10000]['data']['records'][0]['postId']

In [31]:

allPosts = []
videoName = []
for p in posts:
    postJson = postDir + p
    f = open(postJson,'r')
    data = json.load(f)
    allPosts.append(data)
    print data
    videoUrl = data['data']['records'][0]['videoDashUrl']
    if isinstance(videoUrl, basestring):
        videoName.append(videoUrl.split('?')[0].split('/')[-1])

{u'code': u'', u'data': {u'count': 1, u'anchorStr': u'', u'records': [{u'liked': 0, u'videoDashUrl': u'http://v.cdn.vine.co/r/videos_h264dash/32D023AAED1267011278569140224_18655b9ff29.3.2.mp4?versionId=9qDkgrR9_HF9tVUtrOBmPsR8YphU6kLx', u'foursquareVenueId': u'', u'userId': 1147818794988052480, u'private': 0, u'videoWebmUrl': None, u'loops': {u'count': 70537.0, u'velocity': 0.0, u'onFire': 0}, u'thumbnailUrl': u'http://v.cdn.vine.co/r/thumbs/861BC7B18C1267011289663287296_1.3.2.mp4.jpg?versionId=G9I3DcADFrSIbT9MlR3QYJOP_6SUJVyS', u'explicitContent': 0, u'description': u'self made millionaire', u'verified': 0, u'avatarUrl': u'http://v.cdn.vine.co/r/avatars/EA6CAF29461339831587554144256_5bfdb5b11a3.15.0.jpg?versionId=7gA0I0xrN6hgWFiZhUB_bu3thjRqES44', u'comments': {u'count': 166, u'anchorStr': u'1325293486148333568', u'records': [{u'username': u'haaaal', u'comment': u"I've been searching for this song forever. Shazam & looking up the lyrics. I found the lyrics but no song matches it", u'v

TypeError: string indices must be integers

path =  srcRoot + postList[1]
print path
os.path.lexists(path.strip())

In [ ]:
# allVideos = ["" for x in range(len(allPosts))]
# for i in range(len(allPosts)):
#     allVideos[i] = ""
#     if isinstance(allPosts[i]['data'],dict) and isinstance(allPosts[i]['data']['records'][0],dict):
#         videoUrl = allPosts[i]['data']['records'][0]['videoDashUrl']
#         if isinstance(videoUrl, basestring):
#             allVideos[i] = videoUrl.split('?')[0].split('/')[-1]
    
# print len(allVideos)

print len(postList)
for src in postList:
    path =  srcRoot + src
    copyFile(path.strip(),postDir)

In [ ]:
# userList = getList(users)

In [ ]:
# users = []
# for u in userList:
#     fileName = u.split('/')[-1]
#     users.append(fileName.split('.')[0])

In [ ]:
# print users

In [ ]:
#updateVisited(users, visitedList)


print userList[10]
path =  srcRoot + userList[10]
print path
os.path.lexists(path.strip())

print len(userList)
for src in userList:
    path =  srcRoot + src
    copyFile(path.strip(),userDir)

In [ ]:
# videoList = getList(videos)

print videoList[1]

path =  srcRoot + videoList[1]
print path
os.path.exists(path.strip())

print len(videoList)
for src in videoList:
    path =  srcRoot + src
    copyFile(path.strip(),videoDir)

In [ ]:
# print userList[1]

In [ ]:
# savedVids = []
# for v in videoList:
#     savedVids.append(v.split('/')[-1])

In [ ]:
# savedPosts = []
# failedPosts = []
# for vid in savedVids:
#     try:
#         index = allVideos.index(vid)
#         savedPosts.append(allPostNames[index])
#     except ValueError:
#         failedPosts.append(vid)
#         #print "Threw value error at %s" %vid
        
    

In [ ]:
# savedPostIds = []
# for p in savedPosts:
#     savedPostIds.append(p.split('.')[0])
# print savedPostIds[0]

In [ ]:
# print len(videoList)

In [ ]:
# for p in savedPosts: 
#     srcFile = (postDir + p).strip()
#     copyFile(srcFile, savedPostsDir)

In [ ]:
# allPosts = []
# for p in savedPosts:
#     srcFile = (savedPostsDir + p).strip()
#     f = open(srcFile,'r')
#     data = json.load(f)
#     allPosts.append(data)

In [ ]:
# print allPosts[1]